In [1]:
import os
import pandas as pd
import scanpy as sc

from arboreto.algo import grnboost2, genie3
from arboreto.utils import load_tf_names
from distributed import Client, LocalCluster

In [2]:
# read in expression matrix 
# set a working directory
wdir = "/lustre/groups/ml01/workspace/samantha.bening/Bachelor/"
os.chdir( wdir )

adata = sc.read_h5ad('data2/veo_ibd_balanced.h5ad')
adata

AnnData object with n_obs × n_vars = 176789 × 28162
    obs: 'sampID', 'LMUSampID', 'PatientID', 'AnatomicLocation', 'PoolID', 'Protocol', 'CellsLoaded', 'FreezeProtocol', 'AnatomicLocation_Relation', 'AgeYM', 'Sex', 'Inflammation', 'Viability', 'SampleDate', 'sample_LMU_info_Timepoint', 'Ischemia_time', 'n_counts', 'log_counts', 'n_genes', 'mt_frac', 'ribo_frac', 'hb_frac', 'celltype_l3', 'celltype_l1', 'celltype_l2', 'PatientID_genotype'
    uns: 'PatientID_colors', 'PatientID_genotype_colors', 'celltype_l1_colors', 'celltype_l2_colors', 'celltype_l3_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'unintegrated_X_umap'
    varm: 'PCs'
    layers: 'raw'
    obsp: 'connectivities', 'distances'

In [3]:
adata.obs['celltype_l1'].value_counts()

celltype_l1
T Cell        51695
Stroma        51434
Epithelium    33546
B Cell        31015
Myeloid        8049
               1050
Name: count, dtype: int64

In [3]:
# filter for genes not expressed in e.g. 30 or more cells
sc.pp.filter_genes(adata, min_cells=30)
# make expression matrix 
ex_matrix = adata.to_df()
ex_matrix

,AL627309.1,AL627309.3,AL669831.5,FAM87B,LINC00115,FAM41C,AL645608.2,LINC02593,SAMD11,NOC2L,...,MAFIP,AC011043.1,AL592183.1,AC007325.4,AC007325.2,AL354822.1,AC004556.1,AC233755.2,AC233755.1,AC240274.1
CID006480-1:ACCTGTCGTGGTCCCA,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.088189,...,0.0,0.0,0.088189,0.0,0.0,0.0,0.0,0.0,0.0,0.088189
CID006480-1:GTAGTACTCGAGCCTG,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.149761,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
CID006480-1:GTGTGATGTGAGTTTC,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.144580,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
CID006480-1:TTGGATGTCATCGTAG,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.098530,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
CID006480-1:ACACCAACAAATCGGG,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.431547,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CID008847-1:TCGCTCATCATTGCGA,0.0,0.0,0.0,0.0,0.316601,0.0,0.0,0.0,0.0,0.316601,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
CID008847-1:TTCAGGATCGTTCCTG,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
CID008847-1:CCGTTCAAGCTAGCCC,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
CID008847-1:GTCCTCACAAGCACAG,0.0,0.0,0.0,0.0,0.560699,0.0,0.0,0.0,0.0,0.560699,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [4]:
# make 'tf_list' which in this case is just the genes themselves (for gene-gene adjacencies)

tf_names = load_tf_names("../data/allTFs_human.txt")
len(tf_names)

1892

##  Create custom dask client

In [5]:
local_cluster = LocalCluster(n_workers=29, 
                             threads_per_worker=10) # put in the number of cores you gave the job

custom_client = Client(local_cluster)
custom_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 29
Total threads: 290,Total memory: 300.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33683,Workers: 29
Dashboard: http://127.0.0.1:8787/status,Total threads: 290
Started: Just now,Total memory: 300.00 GiB
Comm: tcp://127.0.0.1:46333,Total threads: 10
Dashboard: http://127.0.0.1:35211/status,Memory: 10.34 GiB
Nanny: tcp://127.0.0.1:42311,


## Run GRNBoost2

In [6]:
%%time
network = grnboost2(expression_data=ex_matrix,
                    tf_names=tf_names,
                    client_or_address=custom_client)

/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/lib/python3.12/site-packages/distributed/client.py:3169: UserWarning: Sending large graph of size 15.85 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


2024-03-30 12:51:54,035 - distributed.protocol.core - CRITICAL - Failed to Serialize
Traceback (most recent call last):
  File "/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/lib/python3.12/site-packages/distributed/protocol/core.py", line 109, in dumps
    frames[0] = msgpack.dumps(msg, default=_encode_default, use_bin_type=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/lib/python3.12/site-packages/msgpack/__init__.py", line 36, in packb
    return Packer(**kwargs).pack(o)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "msgpack/_packer.pyx", line 294, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 300, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 297, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 264, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 231, in msgpack._cmsgpack.Packer._pack
  Fil

CancelledError: finalize-1ee4c30438bd043bbe7a9b628d803fbb

In [ ]:
# filter for only importance >= 0.001 
network = network[network['importance'] >= 0.001]

network.to_csv(f'src/SCENICfiles/adj_full_TFs.csv',  header=False, index=False)

In [8]:
custom_client.shutdown()
local_cluster.close()
